In [38]:
from sklearn.datasets import fetch_california_housing

In [39]:
california = fetch_california_housing()

In [40]:
x = california.data
y = california.target

In [41]:
x.shape, y.shape

((20640, 8), (20640,))

In [42]:
x[1].shape

(8,)

In [43]:
## 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras

In [44]:
## 전처리 : train_test_split

from sklearn.model_selection import train_test_split

In [45]:
x_train, x_test, y_train, y_test =\
    train_test_split(x, y, random_state=2022, test_size=0.2)

In [46]:
x_train.shape, y_train.shape

((16512, 8), (16512,))

In [10]:
## 강사 실수!
## 전처리 : min-max scaling

In [11]:
import pandas as pd

In [12]:
pd.DataFrame(x, columns=california.feature_names).describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [13]:
x_train.max(), x_train.min()

max_n = x_train.max()
min_n = x_train.min()

In [14]:
x_train = (x_train - min_n) / (max_n - min_n)
x_test = (x_test - min_n) / (max_n - min_n)

In [15]:
x_train.max(), x_train.min()

(1.0, 0.0)

In [16]:
pd.DataFrame(x_train, columns=california.feature_names).describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,0.003581,0.004274,0.003624,0.003503,0.043359,0.003556,0.004468,0.000134
std,0.000053,0.000352,0.000065,0.000013,0.032062,0.000134,0.000059,0.000056
min,0.003487,0.003501,0.003496,0.003482,0.003557,0.003492,0.004382,0.000000
25%,0.003545,0.003976,0.003597,0.003501,0.025452,0.003541,0.004420,0.000071
50%,0.003572,0.004283,0.003619,0.003502,0.036023,0.003552,0.004429,0.000164
75%,0.003606,0.004506,0.003642,0.003504,0.051760,0.003565,0.004526,0.000177
max,0.003892,0.004925,0.007174,0.004424,1.000000,0.020222,0.004644,0.000280


In [47]:
## 모델링 : Sequential API

# 1번. 세션 클리어
keras.backend.clear_session()

# 2번. 모델 발판 생성
model = keras.models.Sequential()

# 3번. 모델 블록 조립
model.add( keras.layers.Input(shape=(x.shape[1])) )
model.add( keras.layers.Dense(32, activation='relu') )
model.add( keras.layers.Dense(32, activation='relu') )
model.add( keras.layers.Dense(1) )

# 4번. 컴파일 : learning_rate
model.compile(loss='mse', optimizer=keras.optimizers.Adam(0.001) )

# 양념
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,377
Trainable params: 1,377
Non-trainable params: 0
_________________________________________________________________


In [48]:
## Early stopping

from tensorflow.keras.callbacks import EarlyStopping

In [49]:
es = EarlyStopping(monitor='val_loss',           # 무엇을 관찰할래
                   min_delta=0,                  # 0이면, 나빠지지만 않아도 괜찮아
                   patience=5,                   # 5번까지 참을래
                   verbose=1,                 
                   restore_best_weights=True)    # 학습 최적 가중치를 모델에 전달

In [50]:
## 학습 : validation_split, batch_size

model.fit(x_train, y_train, epochs=1000, verbose=1, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/1000
104/104 [==============================] - 0s 1ms/step - loss: 11816.1807 - val_loss: 45.5562
Epoch 2/1000
104/104 [==============================] - 0s 680us/step - loss: 38.1481 - val_loss: 14.5779
Epoch 3/1000
104/104 [==============================] - 0s 699us/step - loss: 5.9986 - val_loss: 2.8012
Epoch 4/1000
104/104 [==============================] - 0s 709us/step - loss: 3.1869 - val_loss: 2.6034
Epoch 5/1000
104/104 [==============================] - 0s 699us/step - loss: 2.9726 - val_loss: 2.5079
Epoch 6/1000
104/104 [==============================] - 0s 699us/step - loss: 2.8317 - val_loss: 2.3326
Epoch 7/1000
104/104 [==============================] - 0s 709us/step - loss: 2.6633 - val_loss: 2.2888
Epoch 8/1000
104/104 [==============================] - 0s 709us/step - loss: 2.5236 - val_loss: 2.1009
Epoch 9/1000
104/104 [==============================] - 0s 699us/step - loss: 2.3751 - val_loss: 1.9962
Epoch 10/1000
104/104 [==============================] - 0s

In [51]:
## 예측

y_pred = model.predict(x_test)

y_pred[:5].reshape(-1)

array([4.4344873, 1.7776269, 1.9468697, 1.7813958, 3.094308 ],
      dtype=float32)

In [52]:
y_test[:5].reshape(-1)

array([4.771, 1.371, 1.233, 0.938, 3.26 ])